# Querry data based on ro-Crates

1. This specifically looks at the intermediate steps of the metaGOflow pipeline.
2. `fastap` outputs
3. ...

**Steps:** (for each metaGOflow step)
1. Acess ro-crate metadata file and extract needed data sources
2. Get the data
3. Visualize

In [ ]:
import requests
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import init_setup
init_setup()

## Imports

In [ ]:
# This needs to be repeated here for the Pannel dashboard to work, WEIRD
# TODO: report as possible bug
import sys
import os
import io

# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import panel as pn

from json2txttree import json2txttree

# All low level functions are imported from the momics package
from momics.loader import get_ro_crate_metadata_gh


# Note: This is breaking the panel preview functionality
%load_ext autoreload
%autoreload 2

In [ ]:
# TODO: is this nomenclature the final one?
sample_id = "EMOBON_PiEGetxo_Wa_4"
met_json = get_ro_crate_metadata_gh(sample_id)

In [ ]:
## pretty nice tool
print(json2txttree(met_json))

## Methods

- I do not really want to code parsing of this html
  - TODO: check for some written analyser of fastap

In [22]:

def extract_data_by_name(metadata, name='FASTP analysis of raw sequence data'):
    for d in metadata['@graph']:
        if 'name' in d.keys() and d['name'] == name:
            data = d
            break
    return data


def extract_all_datafiles(metadata):
    datafiles = []
    for d in metadata['@graph']:
        if 'name' in d.keys() and d['@type'] == 'File':
            data_unit = {}
            data_unit['name'] = d['name']
            try:
                # in MB
                data_unit['sizeMB'] = int(int(d['contentSize'])/1e6)
            except KeyError:
                data_unit['sizeMB'] = 'unknown'

            try:
                data_unit['downloadUrl'] = d['downloadUrl']
            except KeyError:
                data_unit['downloadUrl'] = 'unknown'
            datafiles.append(data_unit)
            
    return datafiles

In [23]:
from IPython.display import display, HTML

In [ ]:
# extract all related data files
datafiles = extract_all_datafiles(met_json)
assert len(datafiles) == len([k for k in datafiles if k['sizeMB'] != 'unknown'])
assert len(datafiles) == len([k for k in datafiles if k['downloadUrl'] != 'unknown'])

len(datafiles), len([k for k in datafiles if k['sizeMB'] < 100])

this means I have 50 outputs which I can do something about with size < 50 MB

In [ ]:
data = extract_data_by_name(met_json)
r = requests.get(data['downloadUrl'])
print(r.status_code)

# both look the same, but plots are generated by scripts, will not display here
# display(HTML(r.content.decode('utf-8')))
HTML(r.content.decode('utf-8'))

In [8]:
# questionable for Gcolab, but locally works
import webbrowser
webbrowser.open_new_tab(data['downloadUrl'])